In [1]:
from __future__ import absolute_import
import os
import yaml
import importlib
import warnings
import json

from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

import pathlib
import random
import string
import re
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [2]:
from hls4ml.utils.config import create_config
from hls4ml.converters.keras_to_hls import keras_to_hls, get_supported_keras_layers, register_keras_layer_handler

C:\Users\Jiang\anaconda3\envs\tf\lib\site-packages\hls4ml\converters\__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [3]:
model = load_model('tran_model.h5')

In [4]:
granularity='model'
default_precision='ap_fixed<16,6>'
default_reuse_factor=1

In [5]:
print(granularity.lower())
print(isinstance(model, dict)) # check if is a dictionary

model
False


In [6]:
model_arch = json.loads(model.to_json()) # not a dictionary change to a json file

In [7]:
model_arch['class_name']

'Functional'

In [8]:
keras_layer_config = model_arch['config']['layers'] # then check keras layer is supported or not

In [9]:
keras_layer_config_struc = json.dumps(keras_layer_config, indent=3)
print(keras_layer_config_struc)

[
   {
      "class_name": "InputLayer",
      "config": {
         "batch_input_shape": [
            null,
            500,
            8
         ],
         "dtype": "float32",
         "sparse": false,
         "ragged": false,
         "name": "input_1"
      },
      "name": "input_1",
      "inbound_nodes": []
   },
   {
      "class_name": "TFOpLambda",
      "config": {
         "name": "tf.compat.v1.shape",
         "trainable": true,
         "dtype": "float32",
         "function": "compat.v1.shape"
      },
      "name": "tf.compat.v1.shape",
      "inbound_nodes": [
         [
            "input_1",
            0,
            0,
            {
               "name": null,
               "out_type": "int32"
            }
         ]
      ]
   },
   {
      "class_name": "SlicingOpLambda",
      "config": {
         "name": "tf.__operators__.getitem",
         "trainable": true,
         "dtype": "float32",
         "function": "__operators__.getitem"
      },
      "name":

In [10]:
config = {}

model_config = {}
model_config['Precision'] = default_precision
model_config['ReuseFactor'] = default_reuse_factor
model_config['Strategy'] = 'Latency'
#model_config['Compression'] = False
#model_config['Trace'] = False
config['Model'] = model_config #return config

In [11]:
config_struc = json.dumps(config, indent=3)
print(config_struc)

{
   "Model": {
      "Precision": "ap_fixed<16,6>",
      "ReuseFactor": 1,
      "Strategy": "Latency"
   }
}


In [12]:
#convert_from_keras_model
hls_config=config
output_dir='my-hls-test'
part='xcu250-figd2104-2L-e'
project_name='myproject'
input_data_tb=None
output_data_tb=None
backend='Vivado'

In [13]:
config = create_config(
        output_dir=output_dir,
        project_name=project_name,
        backend=backend,
        part='xcu250-figd2104-2L-e'
        )

In [14]:
config_struc = json.dumps(config, indent=3)
print(config_struc)

{
   "OutputDir": "my-hls-test",
   "ProjectName": "myproject",
   "Backend": "Vivado",
   "XilinxPart": "xcu250-figd2104-2L-e",
   "Board": null,
   "ClockPeriod": 5,
   "IOType": "io_parallel",
   "HLSConfig": {}
}


In [15]:
config['KerasModel'] = model
config['InputData'] = input_data_tb
config['OutputPredictions'] = output_data_tb
config['HLSConfig'] = {}

In [16]:
model_config = hls_config.get('Model', None)

In [17]:
config['HLSConfig']['Model'] = model_config

In [18]:
config_struc = json.dumps(config['HLSConfig'], indent=3)
print(config_struc)

{
   "Model": {
      "Precision": "ap_fixed<16,6>",
      "ReuseFactor": 1,
      "Strategy": "Latency"
   }
}


In [19]:
# return keras_to_hls(config)

In [20]:
layer_list = []
'KerasModel' in config

True

In [21]:
from hls4ml.converters.keras_to_hls import KerasFileReader, KerasModelReader

In [22]:
keras_model = config['KerasModel']
if isinstance(keras_model, str):
    from tensorflow.keras.models import load_model
    keras_model = load_model(keras_model)
model_arch = json.loads(keras_model.to_json())
reader = KerasModelReader(keras_model)

In [23]:
#Map inputs of skipped and split (activation) layers
inputs_map = {}
#Loop through layers
layer_counter = 0
input_layers = None
output_layers = None
layer_config = None

In [24]:
print('Interpreting Model')
layer_config = model_arch['config']['layers']
input_layers = [ inp[0] for inp in model_arch['config']['input_layers'] ]
output_layers = [ out[0] for out in model_arch['config']['output_layers'] ]

Interpreting Model


In [25]:
input_layers

['input_1']

In [26]:
output_layers

['dense_13']

In [27]:
layer_config

[{'class_name': 'InputLayer',
  'config': {'batch_input_shape': [None, 500, 8],
   'dtype': 'float32',
   'sparse': False,
   'ragged': False,
   'name': 'input_1'},
  'name': 'input_1',
  'inbound_nodes': []},
 {'class_name': 'TFOpLambda',
  'config': {'name': 'tf.compat.v1.shape',
   'trainable': True,
   'dtype': 'float32',
   'function': 'compat.v1.shape'},
  'name': 'tf.compat.v1.shape',
  'inbound_nodes': [['input_1', 0, 0, {'name': None, 'out_type': 'int32'}]]},
 {'class_name': 'SlicingOpLambda',
  'config': {'name': 'tf.__operators__.getitem',
   'trainable': True,
   'dtype': 'float32',
   'function': '__operators__.getitem'},
  'name': 'tf.__operators__.getitem',
  'inbound_nodes': [['tf.compat.v1.shape', 0, 0, {'slice_spec': -2}]]},
 {'class_name': 'TFOpLambda',
  'config': {'name': 'tf.range',
   'trainable': True,
   'dtype': 'float32',
   'function': 'range'},
  'name': 'tf.range',
  'inbound_nodes': [['_CONSTANT_VALUE',
    -1,
    0,
    {'limit': ['tf.__operators__.get

In [28]:
# muti-head in the for loop
mha_layer = layer_config[6]
output_shapes = {}
output_shape = None
mha_layer

{'class_name': 'MultiHeadAttention',
 'config': {'name': 'multi_head_attention',
  'trainable': True,
  'dtype': 'float32',
  'num_heads': 4,
  'key_dim': 2,
  'value_dim': 2,
  'dropout': 0.15,
  'use_bias': True,
  'output_shape': None,
  'attention_axes': [1],
  'kernel_initializer': {'class_name': 'GlorotUniform',
   'config': {'seed': None}},
  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
  'kernel_regularizer': None,
  'bias_regularizer': None,
  'activity_regularizer': None,
  'kernel_constraint': None,
  'bias_constraint': None,
  'query_shape': [None, 500, 8],
  'key_shape': [None, 500, 8],
  'value_shape': [None, 500, 8]},
 'name': 'multi_head_attention',
 'inbound_nodes': [[['tf.__operators__.add',
    0,
    0,
    {'value': ['tf.__operators__.add', 0, 0]}]]]}

In [30]:
keras_layer = mha_layer
keras_class = keras_layer['class_name']
keras_class

'MultiHeadAttention'

In [31]:
# layer, output_shape = layer_handlers[keras_class](keras_layer, input_names, input_shapes, reader, config)
input_names = [ inputs_map.get(inp[0], inp[0]) for inp in keras_layer['inbound_nodes'][0] ] # ['tf.__operators__.add']
input_shapes = [[None, 500, 8]] 
reader

In [32]:
config

{'OutputDir': 'my-hls-test',
 'ProjectName': 'myproject',
 'Backend': 'Vivado',
 'XilinxPart': 'xcu250-figd2104-2L-e',
 'Board': None,
 'ClockPeriod': 5,
 'IOType': 'io_parallel',
 'HLSConfig': {'Model': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'Strategy': 'Latency'}},
 'KerasModel': <keras.engine.functional.Functional at 0x20a05c5f220>,
 'InputData': None,
 'OutputPredictions': None}

In [33]:
# converter

In [34]:
import numpy as np

from hls4ml.converters.keras_to_hls import parse_default_keras_layer
from hls4ml.converters.keras_to_hls import keras_handler

In [35]:
reader.get_weights_shape('dense_11','kernel')

[128, 8]

In [36]:
reader.get_weights_shape('multi_head_attention_1','attention_output/kernel')

[4, 2, 8]

In [37]:
reader.get_weights_shape('multi_head_attention_1','attention_output/bias')

[8]

In [38]:
reader.get_weights_shape('multi_head_attention_1','query/kernel')

[8, 4, 2]

In [39]:
reader.get_weights_shape('multi_head_attention_1','value/bias')

[4, 2]

In [40]:
# @keras_handler('MultiHeadAttention')
# def parse_conv1d_layer(keras_layer, input_names, input_shapes, data_reader, config):
# what is the input_input_shape looks like, since it has two argument?
assert('MultiHeadAttention' in keras_layer['class_name'])

layer = parse_default_keras_layer(keras_layer, input_names)
layer

{'name': 'multi_head_attention',
 'class_name': 'MultiHeadAttention',
 'inputs': ['tf.__operators__.add'],
 'data_format': 'channels_last',
 'use_bias': True}

In [60]:
layer['num_heads'] = keras_layer['config']['num_heads']
layer['head_dim_key'] = keras_layer['config']['key_dim']
layer['head_dim_value'] = keras_layer['config']['value_dim']
layer['query_shape'] = keras_layer['config']['query_shape']
layer['key_shape'] = keras_layer['config']['key_shape']
layer['value_shape'] = keras_layer['config']['value_shape']
layer['feature_dim'] = layer['query_shape'][-1]
# seq_length might not be a constant, not including?


# below lines needs to discuss. Should we include those features or not?
layer['dtype'] = keras_layer['config']['dtype'] 

if keras_layer['config']['output_shape']:
    layer['output_shape'] = keras_layer['config']['output_shape']
else:
    layer['output_shape'] = layer['query_shape']

layer['attention_axes'] = keras_layer['config']['attention_axes'] if (keras_layer['config']['attention_axes'][0]==1) else False

if layer['attention_axes'] is False:
    raise Exception('assigning the attention_axe is not currently supported by hls4ml'.format(layer['class_name']))

if not((len(layer['query_shape'])) == 3 and (len(layer['query_shape'])) == 3 and (len(layer['query_shape'])) == 3):
    raise Exception('muti-dimension of feature dim is not currently supported by hls4ml'.format(layer['class_name']))
    
attn_scores_rank = 4 - 1 # weight matrix has shape (Batch,num_head,seq_query,seq_key),   '-1' because Batch is not included in the weight matrix

layer['softmax_axis'] = tuple(range(attn_scores_rank - len(layer['attention_axes']), attn_scores_rank ))



In [66]:
@keras_handler('MultiHeadAttention')
def parse_conv1d_layer(keras_layer, input_names, input_shapes, data_reader, config): # why we need data_reader and config here
    # what is the input_input_shape looks like, since it has three arguments? (value, key, query)
    # assume input_shapes is: [[None, seq, dim]]
    assert('MultiHeadAttention' in keras_layer['class_name'])
    assert(input_shapes[0]==keras_layer['config']['query_shape'])
    
    layer = parse_default_keras_layer(keras_layer, input_names)
    
    layer['num_heads'] = keras_layer['config']['num_heads']
    layer['head_dim_key'] = keras_layer['config']['key_dim']
    layer['head_dim_value'] = keras_layer['config']['value_dim']
    layer['query_shape'] = keras_layer['config']['query_shape']
    layer['key_shape'] = keras_layer['config']['key_shape']
    layer['value_shape'] = keras_layer['config']['value_shape']
    layer['feature_dim'] = layer['query_shape'][-1]
    # seq_length might not be a constant, not including?


    # below lines needs to discuss. Should we include those features or not?
    layer['dtype'] = keras_layer['config']['dtype'] 

    if keras_layer['config']['output_shape']:
        out_shape = keras_layer['config']['output_shape'] # the config output_shape does not include batch and seq, only dim
        layer['output_shape'] = (layer['query_shape'][:2]).extend(out_shape)
    else:
        layer['output_shape'] = layer['query_shape']
        
    output_shape = layer['output_shape']
    
    layer['attention_axes'] = keras_layer['config']['attention_axes'] if (keras_layer['config']['attention_axes'][0]==1) else False

    if layer['attention_axes'] is False:
        raise Exception('assigning the attention_axe is not currently supported by hls4ml'.format(layer['class_name']))

    if not((len(layer['query_shape'])) == 3 and (len(layer['query_shape'])) == 3 and (len(layer['query_shape'])) == 3):
        raise Exception('muti-dimension of feature dim is not currently supported by hls4ml'.format(layer['class_name']))

    attn_scores_rank = 4 # filter matrix has shape (Batch,num_head,seq_query,seq_key),   
                         # should we '-1', since Batch will not included in the weight matrix

    layer['softmax_axis'] = tuple(range(attn_scores_rank - len(layer['attention_axes']), attn_scores_rank ))

    return layer, output_shape

In [67]:
parse_conv1d_layer(keras_layer, input_names, input_shapes, reader, config)

({'name': 'multi_head_attention',
  'class_name': 'MultiHeadAttention',
  'inputs': ['tf.__operators__.add'],
  'data_format': 'channels_last',
  'use_bias': True,
  'num_heads': 4,
  'head_dim_key': 2,
  'head_dim_value': 2,
  'query_shape': [None, 500, 8],
  'key_shape': [None, 500, 8],
  'value_shape': [None, 500, 8],
  'feature_dim': 8,
  'dtype': 'float32',
  'output_shape': [None, 500, 8],
  'attention_axes': [1],
  'softmax_axis': (3,)},
 [None, 500, 8])